# Logistic Regression

(before Multinomial logistic regression)
We want to predict the probability of an input belonging to one of two classes.

---


## Study case : 
Perform logisitc regression to classify the cats and dogs dataset.
#### a) Dataset
- Input : Images of size ?? where a cat or a dog is present
- Output : 0 if the input is a cat, 1 otherwise

#### b) Classifier
Logistic regression in 3 steps :
- p = exp(W^t X + b)
- C = - y.ln(p) - (1-y)(ln(1-p))
- Gradient descent ^^